# Sorting Transitions

In this notebook we look at our stochastic integrations with strong noise and pick out the transition members. We then select the timepoints of each that we feel make up 'the transition'.

Our choice of 'the transition' is made as follows:

- We do some initial cleaning to ensure we only look at proper transitions, namely you have to spend a certain amount of time in each basin.
- For each transition timesereis we pick the nearest point in the timeseries to (0, 0).
- We then look 270 steps previously (based on Arrhenius estimate) and check if you're near the cold point. If yes, you count as a transition for our purposes.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
# from tqdm.notebook import tqdm
import pandas as pd

## Loading Data

In [3]:
data_dir = 'Data/eps0_05/'
alpha_dirs = [data_dir + a +'/' for a  in os.listdir(data_dir)]

In [4]:
plt.rcParams['figure.figsize'] = (20, 10)

## METHOD 1: AIMING FOR THE SADDLE IN A COMPLEX WAY

## $ \alpha = 1.0 $ Transitions

### Cold to Hot Transitions

In [5]:
wd = alpha_dirs[0] + 'cold-ensemble/'
print(f'Opening files from: \'{wd}\'')

raw_files = []
for f in os.listdir(wd):
    if f[0].isdigit(): # Raw data have numbers as names
        raw_files.append(wd + f)

Opening files from: 'Data/eps0_05/alpha_1_0/cold-ensemble/'


### Step 1: Identify all members of our ensemble integration that feature a transition

In [6]:
# Open all ensemble members in one go, the 'realisation' index conflicts (i.e. all labelled 1 - 50), so we reset it
ds = xr.open_mfdataset(raw_files, concat_dim = 'realisation', combine='nested').reset_index('realisation') 

In [7]:
# Function for picking out transition members
# Transition 'happened' if you spend 1000 steps in each basin (defined as the abs(x) = 0.5 lines)

def transition_members(ensemble_ds): 
    n = 1000
    cold_condition = ((ensemble_ds.x < -0.5).sum(dim='time') > n).values # Realisations with n steps in cold basin
    hot_condition = ((ensemble_ds.x > 0.5).sum(dim='time') > n).values # Realisations with n steps in hot basin
    
    return ensemble_ds.isel(realisation = hot_condition * cold_condition)

In [8]:
t_ds = transition_members(ds)
t_ds.x.plot()
plt.title('Raw Tranistion Members')
plt.show()

KeyboardInterrupt: 

### Step 2: Some Clean Up Ensuring We End Up in The Hot Basin (i.e. don't transition back and forth)

In [ ]:
t_ds = t_ds.isel(realisation = (t_ds.x[:, -1] > 0).values)

In [ ]:
t_ds.x.plot()
plt.title('Tranistion Members Ending in the Hot Basin')
plt.show()

### Step 3: Saving Transition Members as a Single Dataset (before aligning endpoint)

In [ ]:
save_name = wd + 'Raw-Transition-Members.nc'

# Sorting out the realisation coordinates and saving
t_ds = t_ds.drop('realisation_').reindex({'realisation': np.arange(1, 1 + len(t_ds.realisation))}) # Note, this throws error if already run
# t_ds.to_netcdf(save_name)
# print(f'Saved transitions at {save_name}')

### Step 4: Aligning the End Point

In [ ]:
# saddle_indices = np.abs(t_ds.x).argmin('time').values

# Identify the Endpoint as those closest to the saddle
saddle_indices = np.sqrt((t_ds.x **2 + t_ds.y **2)).argmin('time').values


# Fetch the x and y transiton timeseries

transition_len = 270 
transition_time = np.arange(0, transition_len) * 0.01
transition_das = [] # List of transition data arrays

# Loop through Realisations
for k in range(len(saddle_indices)): 
    si = saddle_indices[k]
    
    # Transition timeseries are those in time (si - 270, si) where si is the time you're at (0, 0)
    tts = t_ds.isel(realisation=k).drop('realisation').isel(time = slice(si - transition_len, si))
    
    # Check the transition started near the cold point (-1, 0)
    if ( np.sqrt( (tts.x[0] + 1)**2 + (tts.y[0])**2 ) < 0.1): 
        transition_das.append(tts.reset_index('time', drop=True).assign_coords(time=transition_time))

In [ ]:
clean_transition_ds = xr.concat(transition_das, pd.Index(np.arange(1, len(transition_das) + 1), name='realisation'))
clean_transition_ds.x.plot()
plt.show()
plt.plot(clean_transition_ds.x.mean(dim='realisation'), clean_transition_ds.y.mean(dim='realisation'))

In [ ]:
# Creating a Clean Transition Dataset which we save

# save_name = wd + 'Clean-Transitions.nc'
# clean_transition_ds.to_netcdf(save_name)
# print(f'Saved transitions at {save_name}')

# Method 2: Aiming for the Opposite Minima

## $ \alpha = 1.0 $ Transitions

### Cold to Hot Transitions

In [ ]:
wd = alpha_dirs[0] + 'hot-ensemble/'
print(f'Opening files from: \'{wd}\'')

raw_files = []
for f in os.listdir(wd):
    if f[0].isdigit(): # Raw data have numbers as names
        raw_files.append(wd + f)

### Step 1: Identify all members of our ensemble integration that feature a transition

In [9]:
# Open all ensemble members in one go, the 'realisation' index conflicts (i.e. all labelled 1 - 50), so we reset it
ds = xr.open_mfdataset(raw_files, concat_dim = 'realisation', combine='nested').reset_index('realisation') 

In [10]:
ds

<xarray.Dataset>
Dimensions:       (realisation: 5000, time: 50000)
Coordinates:
  * time          (time) float64 0.0 0.01 0.02 0.03 ... 500.0 500.0 500.0 500.0
    realisation_  (realisation) int64 1 2 3 4 5 6 7 ... 495 496 497 498 499 500
Dimensions without coordinates: realisation
Data variables:
    x             (realisation, time) float64 dask.array<chunksize=(500, 50000), meta=np.ndarray>
    y             (realisation, time) float64 dask.array<chunksize=(500, 50000), meta=np.ndarray>
Attributes:
    alpha:    1.0
    eps:      0.05

In [ ]:
# Function for picking out transition members
# Transition 'happened' if you spend 1000 steps in each basin (defined as the abs(x) = 0.5 lines)

def transition_members(ensemble_ds): 
    n = 100
    cold_condition = ((ensemble_ds.x < -0.5).sum(dim='time') > n).values # Realisations with n steps in cold basin
    hot_condition = ((ensemble_ds.x > 0.5).sum(dim='time') > n).values # Realisations with n steps in hot basin
    
    return ensemble_ds.isel(realisation = hot_condition * cold_condition)

In [ ]:
t_ds = transition_members(ds)
t_ds.x.plot()
plt.title('Raw Tranistion Members')
plt.show()

## Step 2: Identify sequence starting in cold point and exiting in hot point (without re-entering cold_point)

In [ ]:
# Initialise Lists to store results
transitions = []
fishy_indices = []

for k in tqdm(range(len(t_ds.realisation))): # Loop through realisations

    # Get single realisation

    test_data = t_ds.isel(realisation = k).drop('realisation_')

    ball_size = 0.1

    # Assign cold points a -1, assign hot points a 1 
    
    cold_points = -(np.sqrt((test_data.x + 1 )**2 + test_data.y **2) < ball_size).values.astype(int)
    hot_points = (np.sqrt((test_data.x - 1 )**2 + test_data.y **2) < ball_size).values.astype(int)
    symbolic_path = cold_points + hot_points

    # Find the first hot point, i.e. first appearance of a 1

    hot_entry_time = np.argmax(symbolic_path) + 1

    # Find the last cold point prior to hot_time
    
    cold_exit_time = hot_entry_time - np.argmin(np.flip(symbolic_path[:hot_entry_time])) - 1
    
    # Pick out the Transition Path 
    
    transition_path = test_data.isel(time=slice(cold_exit_time, hot_entry_time))
    
    # Double check transition_path only has one point in each attractor
    
    cold_points = -(np.sqrt((transition_path.x + 1 )**2 + transition_path.y **2) < ball_size).values.astype(int)
    hot_points = (np.sqrt((transition_path.x - 1 )**2 + transition_path.y **2) < ball_size).values.astype(int)
    
    if ((np.sum(hot_points) == 1) and (np.sum(cold_points) == -1)):
        transitions.append(transition_path)

    else:
        print('Something Wrong')
        fishy_indices.append(k)

In [ ]:
def _tag_cold_points(ts, ball_size=0.1):
    return -(np.sqrt((ts.x + 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def _tag_hot_points(ts, ball_size=0.1):
    return -(np.sqrt((ts.x - 1 )**2 + ts.y **2) < ball_size).values.astype(int)

def symbolic_ts(ts, ball_size=0.1):
    cold_points = _tag_cold_points(ts, ball_size=0.1)
    hot_points = _tag_cold_points(ts, ball_size=0.1)
    return cold_points + hot_points

## Step 3: Save Results

In [ ]:
# Save individual transitions in a single folder
save_dir = wd + 'Transition-Data/' 

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f'Made directory: {save_dir}')

for k, t in tqdm(enumerate(transitions)):
    save_name = save_dir + f'{k + 1}.nc'
    t.to_netcdf(save_name)

print(f'Saved {len(transitions)} transitions at {save_dir}')

In [11]:
wd

'Data/eps0_05/alpha_1_0/cold-ensemble/'